<a href="https://colab.research.google.com/github/Antoniopv7/Mineria-Prueba2/blob/main/lacopia%20de%20la%20copia%20de%20la%20copia%20de%20la%20copia%20de%20la%20copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importar librerías
import pandas as pd
import urllib.request
import zipfile
import tempfile
import numpy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# 1. Descargar la encuesta CASEN 2022
STATA_zip_url = 'https://observatorio.ministeriodesarrollosocial.gob.cl/storage/docs/casen/2022/Base%20de%20datos%20Casen%202022%20STATA_18%20marzo%202024.dta.zip'
STATA_file_name = 'Base de datos Casen 2022 STATA_18 marzo 2024.dta'

STATA_temp_file = tempfile.NamedTemporaryFile(delete=False)
STATA_zip_name = STATA_temp_file.name
urllib.request.urlretrieve(STATA_zip_url, STATA_zip_name)
with zipfile.ZipFile(STATA_zip_name, 'r') as zip_ref:
    zip_ref.extract(STATA_file_name)

data_STATA = pd.read_stata(STATA_file_name, convert_categoricals=False)

# 2. Formar el dataset, normalizando los valores entre 0 a 1
scaler = MinMaxScaler()
data_STATA_cleaned_normalized = pd.DataFrame(scaler.fit_transform(data_STATA_cleaned), columns=data_STATA_cleaned.columns)

# 3. Aplique transformaciones de los datos para ajustar mejor el modelo de regresión lineal múltiple
# (Aquí puedes explorar diferentes transformaciones como logaritmo, raíz cuadrada, etc.)

# 5. Realice una limpieza de los datos, eliminando valores nulos y outliers
data_STATA_cleaned = data_STATA.dropna(subset=['ytrabajocor'])
data_STATA_cleaned = data_STATA_cleaned.apply(pd.to_numeric, errors='coerce')
data_STATA_cleaned = data_STATA_cleaned.dropna(thresh=0.5 * len(data_STATA_cleaned), axis=1)
data_STATA_cleaned = data_STATA_cleaned.fillna(data_STATA_cleaned.mean())

# 4. Seleccione al menos 8 variables importantes para su modelo y explicando claramente su significado
X = data_STATA_cleaned_normalized.drop('ytrabajocor', axis=1)
y = data_STATA_cleaned_normalized['ytrabajocor']
rfe = RFE(estimator=LinearRegression(), n_features_to_select=8)
X_rfe = rfe.fit_transform(X, y)
columnas_seleccionadas = X.columns[rfe.get_support()]
print("Variables seleccionadas:", columnas_seleccionadas)

# 6. Realice un análisis descriptivo, mostrando al menos los gráficos de dispersión, histogramas, gráficos de torta, caja, etc.
# Análisis descriptivo de las variables seleccionadas
fig, axs = plt.subplots(2, 4, figsize=(16, 8))
axs = axs.flatten()

for i, var in enumerate(columnas_seleccionadas):
    axs[i].hist(data_STATA_cleaned[var], bins=20)
    axs[i].set_title(f'Histograma de {var}')

plt.tight_layout()
plt.show()

fig, axs = plt.subplots(2, 4, figsize=(16, 8))
axs = axs.flatten()

for i, var in enumerate(columnas_seleccionadas):
    axs[i].boxplot(data_STATA_cleaned[var])
    axs[i].set_title(f'Diagrama de caja de {var}')

plt.tight_layout()
plt.show()

# 7. Obtenga el mejor ajuste posible con al menos un R^2 igual a 0.7
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R^2: {r2:.2f}")

# 8. Compruebe que su modelo satisface las condiciones de regresión lineal múltiple (Hipótesis de Normalidad)
residuos = y_test - y_pred
sm.qqplot(residuos, line='s')
plt.title("Gráfico de Normalidad de Residuos")
plt.show()

# 9. Calcule la matriz de correlaciones entre las variables e interprete, realice el gráfico de correlaciones entre los pares de variables
correlation_matrix = data_STATA_cleaned_normalized.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="YlGnBu")
plt.title("Matriz de Correlaciones")
plt.show()

# 10. Represente el modelo de regresión lineal múltiple interpretando y explicando la importancia de cada uno de los coeficientes betas
coef = pd.Series(model.coef_, index=X.columns[rfe.get_support()])
print("Coeficientes del modelo:")
print(coef)

# 11. Calcule al menos 2 índices de error de la RLM
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"Raíz del error cuadrático medio (RMSE): {rmse:.2f}")

# 12. Realice una predicción basado en su modelo
nuevos_datos = pd.DataFrame(columns=X.columns[rfe.get_support()])
nuevos_datos.loc[0] = [0.5, 0.6, 0.7, 0.8, 0.9, 0.2, 0.3, 0.4]  # Reemplazar con los valores correspondientes
predicciones = model.predict(scaler.transform(nuevos_datos))
print("Predicciones:", predicciones)

# 13. Realice un gráfico de la recta de RLM sobre los datos
plt.figure(figsize=(10, 6))
plt.scatter(X_test[:, 0], y_test, label='Datos reales')
plt.plot(X_test[:, 0], y_pred, color='red', label='Regresión lineal múltiple')
plt.xlabel(X_rfe.columns[0])
plt.ylabel('Variable objetivo')
plt.title('Gráfico de regresión lineal múltiple')
plt.legend()
plt.show()

# 15. Discuta cómo se podría mejorar este modelo
print("Posibles mejoras del modelo:")
print("- Explorar transformaciones de variables para mejorar la linealidad")
print("- Considerar modelos no lineales (ej. regresión polinomial, árboles de decisión)")
print("- Realizar una validación cruzada para una mejor estimación del rendimiento")
print("- Ajustar los hiperparámetros del modelo (ej. regularización)")
print("- Probar técnicas de ensemble (ej. Random Forest, Gradient Boosting)")

<ipython-input-5-ec26da2397b7>:26: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  data_STATA = pd.read_stata(STATA_file_name, convert_categoricals=False)
